# Module 2 Assessment

Welcome to your Mod 2 Assessment. You will be tested for your understanding of concepts and ability to solve problems that have been covered in class and in the curriculum.

Use any libraries you want to solve the problems in the assessment.

You will have up to two hours to complete this assessment.

The sections of the assessment are:

- Accessing Data Through APIs
- Object Oriented Programming
- SQL and Relational Databases
- HTML, CSS and Web Scraping
- Other Database Structures (MongoDB)

In this assessment you will be exploring two datasets: Pokemon and Quotes.

In [2]:
# import the necessary libraries

import requests
import json
import pandas as pd
import sqlite3
from bs4 import BeautifulSoup
import pymongo

    100% |████████████████████████████████| 317kB 8.7MB/s eta 0:00:01


## Part 1: Accessing Data Through APIs

In this section we'll be using PokeAPI to get data on Pokemon. Let's first define functions to get information from the API. Provided below is a URL that will get you started with the first 151 Pokemon! Run the cell below to see what we get.

In [47]:
url = 'https://pokeapi.co/api/v2/pokemon/?limit=151'
for i in range(0,151):
    results = requests.get(url).json()['results']


[Read the documentation here](https://pokeapi.co/) for information on navigating this API and use the API to obtain data to answer the following questions.

### Accessing Data

1. For any **one** Pokemon, retrieve the following information in a dictionary format with the following keys:
    - ID
    - Name
    - Base experience
    - Weight
    - Height
    - Types
    - Abilities

For `Types` and `Abilities`, you might want to write helper functions to have each attribute just be a list of types and a list of abilities. Your output should look like this:

```
{'id': 1, 
'name': 'bulbasaur', 
'base_experience': 64, 
'weight': 69, 
'height': 7, 
'types': ['poison', 'grass'], 
'abilities': ['chlorophyll', 'overgrow']}

```
    


In [48]:
# you may define helper functions for types and abilities here
def get_abilities(row):
    return [ability['ability']['name'] for ability in row['abilities']]

def get_types(row):
    return[poke_type['type']['name'] for poke_type in row['types']]

In [55]:

def get_pokedata(url):   
    """
    url is a string of the URL that will get you the relevant info from the API
    
    this function should return the dictionary for ONE pokemon
    
    """
    response = requests.get(url)
    data = response.json()
    return {
        'ID':data['id'],
        'name':data['name'],
        'base_experience':data['base_experience'],
        'weight': data['weight'],
        'height': data['height'],
        'types': get_types(data),
        'abilities': get_abilities(data)
    }


### Pagination

2. Get the same information for the first **151** Pokemon as a list of dictionaries ordered by Pokemon ID. Print the first and last elements of the list. (Hint: Use pagination) Your output should save the list to a variable and look like this:

```
[{'id': 1, 
'name': 'bulbasaur', 
'base_experience': 64, 
'weight': 69, 
'height': 7, 
'types': ['poison', 'grass'], 
'abilities': ['chlorophyll', 'overgrow']}, 
{'id': 2, 
'name': 'ivysaur', 
'base_experience': 142, 
'weight': 130, 
'height': 10, 
'types': ['poison', 'grass'], 
'abilities': ['chlorophyll', 'overgrow']}, ... ]

```



In [37]:
requests.get(url).json()

{'count': 964,
 'next': 'https://pokeapi.co/api/v2/pokemon/?offset=151&limit=151',
 'previous': None,
 'results': [{'name': 'bulbasaur',
   'url': 'https://pokeapi.co/api/v2/pokemon/1/'},
  {'name': 'ivysaur', 'url': 'https://pokeapi.co/api/v2/pokemon/2/'},
  {'name': 'venusaur', 'url': 'https://pokeapi.co/api/v2/pokemon/3/'},
  {'name': 'charmander', 'url': 'https://pokeapi.co/api/v2/pokemon/4/'},
  {'name': 'charmeleon', 'url': 'https://pokeapi.co/api/v2/pokemon/5/'},
  {'name': 'charizard', 'url': 'https://pokeapi.co/api/v2/pokemon/6/'},
  {'name': 'squirtle', 'url': 'https://pokeapi.co/api/v2/pokemon/7/'},
  {'name': 'wartortle', 'url': 'https://pokeapi.co/api/v2/pokemon/8/'},
  {'name': 'blastoise', 'url': 'https://pokeapi.co/api/v2/pokemon/9/'},
  {'name': 'caterpie', 'url': 'https://pokeapi.co/api/v2/pokemon/10/'},
  {'name': 'metapod', 'url': 'https://pokeapi.co/api/v2/pokemon/11/'},
  {'name': 'butterfree', 'url': 'https://pokeapi.co/api/v2/pokemon/12/'},
  {'name': 'weedle', 

In [56]:
""" 

Assign to "pokedata" the list of 151 dictionaries.
You may use your function from the previous question.

"""
pokedata= []
# Your code here
for pokemon in requests.get(url).json()['results']:
    pokedata.append(get_pokedata(pokemon['url']))



In [52]:
# printing first and last elements

print(pokedata[0], pokedata[-1])

{'id': 1, 'name': 'bulbasaur', 'base_experience': 64, 'weight': 69, 'height': 7, 'types': ['poison', 'grass'], 'abilities': ['chlorophyll', 'overgrow']} {'id': 151, 'name': 'mew', 'base_experience': 270, 'weight': 40, 'height': 4, 'types': ['psychic'], 'abilities': ['synchronize']}


## Part 2: Object Oriented Programming

We're going to use the data gathered in the previous section on APIs for this section on Object Oriented Programming to instantiate Pokemon objects and write instance methods.

### Creating a Class

1. Create a class called `Pokemon` with an `__init__` method to instantiate the following attributes:
    - ID
    - Name
    - Base experience
    - Weight
    - Height
    - Types
    - Abilities
    



In [ ]:
# if you were unable to get the data from the API in the right format,
# uncomment the code below to access a JSON file with the list of dictionaries

# with open('data/pokemon.json') as f:  
#     pokelist = json.load(f)

In [85]:
"""
Create your class below with the correct syntax, including an __init__ method.

"""
class Pokemon():
    def __init__(self, ID, name, base_experience, weight, height, types, abilities):
        self.ID = ID
        self.name = name
        self.exp = base_experience
        self.weight = weight
        self.height = height
        self.types = types
        self.abilities = abilities
    
    def bmi(self):
        return round(((self.weight*.1)/(self.height*.1)**2),2)


    
### Instantiating Objects

2. Using the data you obtained from the API, instantiate the first, fourth and seventh Pokemon. Assign them to the variables `bulbasaur`, `charmander` and `squirtle`.

In [86]:
# Your code here

bulbasaur = Pokemon(**pokedata[0])
charmander = Pokemon(**pokedata[3])
squirtle = Pokemon(**pokedata[6])


In [87]:
# run this cell to test and check your code
# you may need to edit the attribute variable names if you named them differently!

def print_pokeinfo(pokemon_object):
    o = pokemon_object
    print('ID: ' + str(o.ID) + '\n' +
          'Name: ' + o.name.title() + '\n' +
          'Base experience: ' + str(o.exp) + '\n' +
          'Weight: ' + str(o.weight) + '\n' +
          'Height: ' + str(o.height) + '\n' +
          'Types: ' + str(o.types) + '\n' +
          'Abilities: ' + str(o.abilities) + '\n'
         )
    
print_pokeinfo(bulbasaur)
print_pokeinfo(charmander)
print_pokeinfo(squirtle)

ID: 1
Name: Bulbasaur
Base experience: 64
Weight: 69
Height: 7
Types: ['poison', 'grass']
Abilities: ['chlorophyll', 'overgrow']

ID: 4
Name: Charmander
Base experience: 62
Weight: 85
Height: 6
Types: ['fire']
Abilities: ['solar-power', 'blaze']

ID: 7
Name: Squirtle
Base experience: 63
Weight: 90
Height: 5
Types: ['water']
Abilities: ['rain-dish', 'torrent']



### Instance Methods

3. Write an instance method within the class `Pokemon` to find the BMI of a Pokemon. BMI is defined by $\frac{weight}{height^{2}}$ with weight in **kilograms** and height in **meters**. The height and weight data of Pokemon from the API is in **decimeters** and **hectograms** respectively.


    1 decimeter = 0.1 meters
    1 hectogram = 0.1 kilograms

In [88]:
# run this cell to test and check your code
# you will probably have to rerun the code to instantiate your objects

print(bulbasaur.bmi()) # 14.08
print(charmander.bmi()) # 23.61
print(squirtle.bmi()) # 36

14.08
23.61
36.0


## Part 3: SQL and Relational Databases

For this section, we've put the Pokemon data into SQL tables. You won't need to use your list of dictionaries or the JSON file for this section. The schema of `pokemon.db` is as follows:

<img src="data/pokemondb.png" alt="db schema" style="width:500px;"/>

Assign your SQL queries as strings to the variables `q1`, `q2`, etc. and run the cells at the end of this section to print your results as Pandas DataFrames.

- q1: query all columns from `Pokemon` the Pokemon that have base_experience above 200  

  
- q2: query the id, name, type1 and type2 of Pokemon that have **water** types as either their first or second type


- q3: query the average weight of Pokemon by their first type in descending order


- q4: query the Pokemon name, Pokemon type2, and what **type2** has "2xdamage" to


- q5: query the top 5 most common type1s, the minimum height, maximum height, minimum weight and maximum weight of pokemon with those type1s, and what associated type they do "0.5xdamage" to


**Important note on syntax**: use `double quotes ""` when quoting strings **within** your query and wrap the entire query in `single quotes ''` For the column titles that begin with numbers, you need to wrap the column names in double quotes.

In [89]:
cnx = sqlite3.connect('data/pokemon.db')

In [90]:
q1 = 'SELECT * FROM pokemon WHERE base_experience > 200'
pd.read_sql(q1, cnx)

,id,name,base_experience,weight,height,type1,type2
0,3,venusaur,236,1000,20,grass,poison
1,6,charizard,240,905,17,fire,flying
2,9,blastoise,239,855,16,water,None
3,18,pidgeot,216,395,15,normal,flying
4,26,raichu,218,300,8,electric,None
5,31,nidoqueen,227,600,13,poison,ground
6,34,nidoking,227,620,14,poison,ground
7,36,clefable,217,400,13,fairy,None
8,45,vileplume,221,186,12,grass,poison
9,62,poliwrath,230,540,13,water,fighting


In [92]:
q2 = 'SELECT id, name, type1, type2 FROM pokemon WHERE "water" in (type1, type2)'
pd.read_sql(q2, cnx)

,id,name,type1,type2
0,7,squirtle,water,None
1,8,wartortle,water,None
2,9,blastoise,water,None
3,54,psyduck,water,None
4,55,golduck,water,None
5,60,poliwag,water,None
6,61,poliwhirl,water,None
7,62,poliwrath,water,fighting
8,72,tentacool,water,poison
9,73,tentacruel,water,poison


In [93]:
q3 = 'SELECT type1, AVG(weight) FROM pokemon GROUP BY type1'
pd.read_sql(q3, cnx)

,type1,AVG(weight)
0,bug,229.916667
1,dragon,766.000000
2,electric,317.888889
3,fairy,237.500000
4,fighting,542.857143
5,fire,480.250000
6,ghost,135.666667
7,grass,279.916667
8,ground,452.625000
9,ice,480.000000


In [95]:
q4 = 'SELECT p.name, type2, "2xdamage" FROM pokemon p JOIN types t ON p.type2 = t.name'
pd.read_sql(q4, cnx)

,name,type2,2xdamage
0,bulbasaur,poison,grass
1,ivysaur,poison,grass
2,venusaur,poison,grass
3,charizard,flying,fighting
4,butterfree,flying,fighting
5,weedle,poison,grass
6,kakuna,poison,grass
7,beedrill,poison,grass
8,pidgey,flying,fighting
9,pidgeotto,flying,fighting


In [96]:
#query the top 5 most common type1s, the minimum height, maximum height, 
#minimum weight and maximum weight of pokemon with those type1s, and what 
#associated type they do "0.5xdamage" to
q5 = 'SELECT type1, MIN(height), MAX(height), MIN(weight), MAX(weight), "0.5xdamage" \
        FROM pokemon p \
        JOIN types t ON p.type1 = t.name \
        GROUP BY type1;'
pd.read_sql(q5, cnx)

,type1,MIN(height),MAX(height),MIN(weight),MAX(weight),0.5xdamage
0,bug,3,15,29,560,fighting
1,dragon,18,40,33,2100,fire
2,electric,3,16,60,666,flying
3,fairy,6,13,75,400,fighting
4,fighting,5,16,195,1300,rock
5,fire,6,20,85,1550,bug
6,ghost,13,16,1,405,poison
7,grass,4,20,25,1200,ground
8,ground,2,19,8,1200,poison
9,ice,14,17,406,554,ice


## Section 4: Web Scraping

### Accessing Data Using BeautifulSoup

Use BeautifulSoup to get quotes, authors, and tags from [Quotes to Read](http://quotes.toscrape.com/).

Before answering these questions, go to the site and inspect the page. Make sure to look at what links there are and how the site is structured.

1. Get the first author and the path for the author's page as a tuple from the [homepage](http://quotes.toscrape.com/).

In [101]:
# Make a get request to retrieve the page
html_page = requests.get('http://quotes.toscrape.com/') 
# Pass the page contents to beautiful soup for parsing
soup = BeautifulSoup(html_page.content, 'html.parser')

# Your code here
author = (soup.find('small',{'class': 'author'}).string, soup.find('small',{'class': 'author'}).next_sibling.next_sibling['href'])
author

('Albert Einstein', '/author/Albert-Einstein')

2. Write a function to get **all** the authors and href links for the authors from the [homepage](http://quotes.toscrape.com/)


In [103]:
def authors(url):
    '''
    input: url
    
    return: a dictionary of of authors and their urls
            {'author_1':'url_of_author_1', 'author_2':'url_of_author_2' ...}
    '''
    # Make a get request to retrieve the page
    html_page = requests.get(url) 
    # Pass the page contents to beautiful soup for parsing
    soup = BeautifulSoup(html_page.content, 'html.parser')
    authors = soup.find_all('small', {'class':'author'})
    out_list = []
    for author in authors:
        out_list.append((author.string, author.next_sibling.next_sibling['href']))
    return out_list

In [104]:
# run this cell to test the function
print(authors('http://quotes.toscrape.com/'))
print()
print()
print(authors('http://quotes.toscrape.com/page/3'))

[('Albert Einstein', '/author/Albert-Einstein'), ('J.K. Rowling', '/author/J-K-Rowling'), ('Albert Einstein', '/author/Albert-Einstein'), ('Jane Austen', '/author/Jane-Austen'), ('Marilyn Monroe', '/author/Marilyn-Monroe'), ('Albert Einstein', '/author/Albert-Einstein'), ('André Gide', '/author/Andre-Gide'), ('Thomas A. Edison', '/author/Thomas-A-Edison'), ('Eleanor Roosevelt', '/author/Eleanor-Roosevelt'), ('Steve Martin', '/author/Steve-Martin')]


[('Pablo Neruda', '/author/Pablo-Neruda'), ('Ralph Waldo Emerson', '/author/Ralph-Waldo-Emerson'), ('Mother Teresa', '/author/Mother-Teresa'), ('Garrison Keillor', '/author/Garrison-Keillor'), ('Jim Henson', '/author/Jim-Henson'), ('Dr. Seuss', '/author/Dr-Seuss'), ('Albert Einstein', '/author/Albert-Einstein'), ('J.K. Rowling', '/author/J-K-Rowling'), ('Albert Einstein', '/author/Albert-Einstein'), ('Bob Marley', '/author/Bob-Marley')]


### Pagination

3. Get the first author on each of the first 5 pages of quotes. You can get to the next page with the next button at the bottom of the homepage.


In [114]:
# Your code here
# Make a get request to retrieve the page
base_url = 'http://quotes.toscrape.com'
quotes_url = '/'
first_authors = []
while len(first_authors) < 6:
    html_page = requests.get(base_url + quotes_url) 
    # Pass the page contents to beautiful soup for parsing
    soup = BeautifulSoup(html_page.content, 'html.parser')
    # Your code here
    first_authors.append(soup.find('small',{'class': 'author'}).string)
    quotes_url = list(soup.find('li', {'class':'next'}).children)[1]['href']
    
print(first_authors)

['Albert Einstein', 'Marilyn Monroe', 'Pablo Neruda', 'Dr. Seuss', 'George R.R. Martin', 'Jane Austen']


4. Write a function to get all of the quotes from a page.

In [130]:
import sys
def get_some_quotes(url, num_pages=1):
    '''
    input: url, number of pages to scrap (just scrape the home page if no argument is passed in)
    
    return: a list of dictionaries of quotes with their attributes
            [{'quote':'quote_1_text', 'author':'url_of_author_1'}, 
            {'quote':'quote_2_text', 'author':'url_of_author_2', 'quote_tags':[list_of_quote_2_tags]}, ...]
    '''
    page_url = '/'
    out_list = []

    # Make a get request to retrieve the page
    for _ in range(num_pages):
        html_page = requests.get(url + page_url) 
        # Pass the page contents to beautiful soup for parsing
        soup = BeautifulSoup(html_page.content, 'html.parser')

        quotes = soup.find_all('span',{'class':'text'})
        authors = soup.find_all('small', {'class':'author'})
        tags = soup.find_all('div', {'class':'tags'})
        for quote, author, tag_group in zip(quotes, authors, tags):
            
            if len(tag_group.text) > 2:
                out_list.append({
                    'quote':quote.string,
                    'author':author.string,
                    'tags': [tag.text for tag in tag_group.find_all('a',{'class':'tag'})]
                })
            else:
                    out_list.append({
                        'quote': quote.string,
                        'author': author.string
                    })
        page_url = list(soup.find('li', {'class':'next'}).children)[1]['href']

    return out_list
print(get_some_quotes('http://quotes.toscrape.com'))

[{'quote': '“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”', 'author': 'Albert Einstein', 'tags': ['change', 'deep-thoughts', 'thinking', 'world']}, {'quote': '“It is our choices, Harry, that show what we truly are, far more than our abilities.”', 'author': 'J.K. Rowling', 'tags': ['abilities', 'choices']}, {'quote': '“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”', 'author': 'Albert Einstein', 'tags': ['inspirational', 'life', 'live', 'miracle', 'miracles']}, {'quote': '“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”', 'author': 'Jane Austen', 'tags': ['aliteracy', 'books', 'classic', 'humor']}, {'quote': "“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”", 'author': 'Marilyn Monroe', 'tags': ['be-yourself', 'inspira

In [131]:
# set the function to a variable to use later
quotes_for_mongo = get_some_quotes('http://quotes.toscrape.com/', 5)
quotes_for_mongo

[{'quote': '“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”',
  'author': 'Albert Einstein',
  'tags': ['change', 'deep-thoughts', 'thinking', 'world']},
 {'quote': '“It is our choices, Harry, that show what we truly are, far more than our abilities.”',
  'author': 'J.K. Rowling',
  'tags': ['abilities', 'choices']},
 {'quote': '“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”',
  'author': 'Albert Einstein',
  'tags': ['inspirational', 'life', 'live', 'miracle', 'miracles']},
 {'quote': '“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”',
  'author': 'Jane Austen',
  'tags': ['aliteracy', 'books', 'classic', 'humor']},
 {'quote': "“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”",
  'author': 'Marilyn Monroe',
  'tags': 

## Part 5: MongoDB

To do this section, open a connection to a mongo database in the terminal, using `mongod`. You will then **create**, **update**, and **read** from a mongo database.

Create and connect to a mongo database.

In [ ]:
myclient = pymongo.MongoClient("mongodb://127.0.0.1:27017/")
mydb = myclient['quote_database']

In [ ]:
mycollection = mydb['quote_collection']

1. Add the quotes you obtained from the `get_some_quotes` function for the [homepage](http://quotes.toscrape.com/) to the mongo database. (You can also use the JSON file `quotes.json` to insert data into the database) To verify that you've successfully inserted the data, query it to obtain the resulting _ids back from the `results` variable. 

In [ ]:
# if you were unable to get the data from webscraping in the right format,
# uncomment the code below to access a JSON file with the list of dictionaries

# with open(r"data/quotes.json", "r") as r:
#     data = json.load(r)

In [ ]:
# use the results variable to confirm the data was inserted
results = None

2. Query the database for all the quotes written by `'Albert Einstein'`.

In [ ]:
q1 = None

3. Update Steve Martin's quote with the tags for the quote stored in the variable `steve_martin_tags`.

In [ ]:
steve_martin_tags = {'quote_tags': ['change', 'deep-thoughts', 'thinking', 'world']}
update_steve = None
first_quote_tags = None


4. Query the database to confirm that  `'Steve Martin'` is updated with `steve_martin_tags`.

In [ ]:
q2 = None